In [1]:
import pandas as pd
import glob
import json
from tqdm import tqdm
import re
from datetime import datetime
import gensim
import logging
import os
import shutil

In [2]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [4]:
df = pd.read_csv("talks.csv.gzip", compression="gzip")

In [ ]:
datetime.strptime('20100929', '%Y%m%d')


In [ ]:
df.head()

In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
?text.TfidfVectorizer

In [ ]:
from sklearn.feature_extraction import text
Text=df['transcript'].tolist()
tfidf=text.TfidfVectorizer(input=Text,stop_words="english", tokenizer=LemmaTokenizer())
matrix=tfidf.fit_transform(Text)
#print(matrix.shape)

In [ ]:
### Get Similarity Scores using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
sim_unigram=cosine_similarity(matrix)

In [ ]:
sim_unigram

In [ ]:
def get_similar_articles(x):
    return "\n".join(df['title'].loc[x.argsort()[-5:-1]])
df['similar_articles_unigram']=[get_similar_articles(x) for x in sim_unigram]

In [ ]:
df['title'][1]

In [ ]:
print (df['similar_articles_unigram'][1])

# Doc2Vec

In [ ]:
df.iloc[0]

In [ ]:
[row["title"]] + row["categories"]

In [ ]:
talk_corpus = []
for (index,row) in df.iterrows():
    content = gensim.utils.simple_preprocess (row["transcript"])
    #tags = [row["title"], row["id"]] + row["categories"]'
    #tags = [row["title"]] + row["categories"]
    tags = [row["title"]] 
    tg = gensim.models.doc2vec.TaggedDocument(content,tags)
    talk_corpus.append(tg)

In [ ]:
talk_corpus[0]

In [ ]:
model = gensim.models.Doc2Vec(size = 300, 
                              min_count = 3, 
                              iter = 100)

In [ ]:
model.build_vocab(talk_corpus)
print("model's vocabulary length:", len(model.wv.vocab))

In [ ]:
model.train(talk_corpus,total_examples=model.corpus_count,epochs=model.epochs)

In [ ]:
model.docvecs.most_similar('My escape from North Korea | Hyeonseo Lee')

In [ ]:
df.head(1)

In [ ]:
content = gensim.utils.simple_preprocess (df.iloc[0]["transcript"])

In [ ]:
content

In [ ]:
new_vector = model.infer_vector(content)
sims = model.docvecs.most_similar([new_vector],topn=20)
sims